In [7]:
import torch 
import numpy as np
import tqdm
import matplotlib.pyplot as plt 
import json

In [8]:
from datasets import load_dataset

In [9]:
gretel_dataset = load_dataset("gretelai/synthetic_text_to_sql")

In [10]:
train = gretel_dataset["train"].to_pandas()
test = gretel_dataset["test"].to_pandas()